In [1]:
# using Pkg;
# #Pkg.add("DFTK")
# Pkg.update()

using ParallelTemperingMonteCarlo
using DelimitedFiles
using StaticArrays
using Random#, Plots

# Pkg.add("BenchmarkTools")
using BenchmarkTools

  ** incremental compilation may be fatally broken for this module **



┌ Warning: Replacing docs for `ParallelTemperingMonteCarlo.MCRun.mc_cycle! :: Tuple{Any, Any, Any, ParallelTemperingMonteCarlo.EnergyEvaluation.AbstractMLPotential, Any, Any, Any, Any, Any}` in module `ParallelTemperingMonteCarlo.MCRun`
└ @ Base.Docs docs/Docs.jl:240
  ** incremental compilation may be fatally broken for this module **

┌ Warning: Replacing docs for `ParallelTemperingMonteCarlo.MCRun.sampling_step! :: Tuple{Any, Any, Any, Bool}` in module `ParallelTemperingMonteCarlo.MCRun`
└ @ Base.Docs docs/Docs.jl:240
  ** incremental compilation may be fatally broken for this module **

┌ Warning: Replacing docs for `ParallelTemperingMonteCarlo.MCRun.save_params :: Tuple{IOStream, ParallelTemperingMonteCarlo.InputParams.MCParams}` in module `ParallelTemperingMonteCarlo.MCRun`
└ @ Base.Docs docs/Docs.jl:240
  ** incremental compilation may be fatally broken for this module **

┌ Warning: Replacing docs for `ParallelTemperingMonteCarlo.MCRun.save_state :: Tuple{IOStream, ParallelTemp

In [3]:
genvector1(max_displacement) = max_displacement * (rand(-0.5:0.001:0.5,3))

genvector2(max_displacement) = SVector((rand()-0.5)*max_displacement,(rand()-0.5)*max_displacement,(rand()-0.5)*max_displacement)
@benchmark genvector1(rand())

BenchmarkTools.Trial: 10000 samples with 585 evaluations.
 Range (min … max):  200.342 ns …   9.600 μs  ┊ GC (min … max): 0.00% … 94.68%
 Time  (median):     256.068 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   282.654 ns ± 292.700 ns  ┊ GC (mean ± σ):  3.61% ±  3.54%

  █▆▅▄▃▃▂▁▇▅▄▃▄▄▃▃▄▂▃▄▃▂▂▁      ▂▂▂▁                            ▂
  █████████████████████████▇█████████▇▇▇██▇███▇█▆▅▆▅▄▆▄▄▅▂▆▄▄▄▄ █
  200 ns        Histogram: log(frequency) by time        591 ns <

 Memory estimate: 160 bytes, allocs estimate: 2.

In [4]:
@benchmark genvector2(rand())

BenchmarkTools.Trial: 10000 samples with 999 evaluations.
 Range (min … max):   9.309 ns … 249.550 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     11.912 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   12.131 ns ±   6.090 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂  ▁        █   ▄                                             
  █▇██▇▄▂▂▁▁▁▅█▄▂▂█▇▂▂▁▁▂▇▂▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  9.31 ns         Histogram: frequency by time           22 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [3]:
n_atoms = 13

# temperature grid
ti = 5.
tf = 16.
n_traj = 32

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details

mc_cycles = 80000 #default 20% equilibration cycles on top

mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)

#ELJpotential for neon
#c1=[-10.5097942564988, 0., 989.725135614556, 0., -101383.865938807, 0., 3918846.12841668, 0., -56234083.4334278, 0., 288738837.441765]
#elj_ne1 = ELJPotential{11}(c1)

c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

#starting configurations
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
bc_ne13 = SphericalBC(radius=5.32*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_ne13, bc_ne13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)

Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[])

Below is the comprehensive testing performed on making a vectorised displacement function

In [7]:
function testtime(mc_states)
    for state in mc_states
        ran = rand(1:length(state.config.pos))
        trial_pos =atom_displacement(state.config.pos[ran],state.max_displ[1],state.config.bc)
    end
end
@benchmark testtime(mc_states)
# rand(1:n_atoms,n_traj)
# V = [1 2 3; 4 5 6; 7 8 9]
# V[:,1]
function atom_disp(mc_state,index)
    trial_pos = atom_displacement(mc_state.config.pos[index],mc_state.max_displ[1],mc_state.config.bc)
    return trial_pos
end
function atom_disp!(mc_state,index,trial_pos::SVector)
    trial_pos = atom_displacement(mc_state.config.pos[index],mc_state.max_displ[1],mc_state.config.bc)
    return trial_pos
end

indices = rand(1:n_atoms,n_traj)

# trial_pos = atom_disp.(mc_states,indices)

@benchmark begin   
    indices = rand(1:n_atoms,n_traj)
    trial_positions = atom_disp.($mc_states,indices)
end

@benchmark begin 
    indices = rand(1:n_atoms,n_traj)
    trial_positions =  [@SVector zeros(3) for i in 1:n_traj]
    atom_disp!.($mc_states,indices,trial_positions)
end
function generate_displacements(mc_states,mc_params)
    indices=rand(1:mc_params.n_atoms,mc_params.n_traj)
    trial_positions = atom_disp.(mc_states,indices)
    return indices,trial_positions
end
@benchmark generate_displacements(mc_states,mc_params)
@benchmark dimer_energy_atom(1, mc_states[1].dist2_mat[1,:], pot)
@benchmark dimer_energy_atom(1, mc_states[1].dist2_mat[:,1], pot)

testtime (generic function with 1 method)

In [4]:
@time ptmc_run!(mc_states, move_strat, mc_params, pot, ensemble, results; save_ham = false)

Total number of moves per MC cycle: 13



equilibration done


MC loop done.


[0.0003213639652679902, 0.00034147319311620696, 0.0003575366210942533, 0.0003717984067306107, 0.0003888641196402448, 0.000401394140720345, 0.000431137601904572, 0.0004701870065195387, 0.0005152291527236873, 0.000516675545297199, 0.0005438275845347618, 0.0006013479597479453, 0.0006157987544912303, 0.0006383368371394352, 0.0007195196982462192, 0.0007798493781618379, 0.0008569887186325168, 0.0009117506744718736, 0.0010427807812447652, 0.0013757374026508987, 0.001686054940826271, 0.0019638227271451788, 0.0023299070417411873, 0.0029999429449483455, 0.0030512014427201265, 0.002955736375726185, 0.0033148809254672793, 0.0027218787424441553, 0.0024728927662745603, 0.0023034849772417556, 0.0020764297127493178, 0.002011915186489072]
done
 29.250604 seconds (90.67 M allocations: 12.896 GiB, 7.70% gc time, 9.27% compilation time)
